In [11]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

stopWords = set(word.strip().lower() for word in open("stop_words_en.txt"))

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        word = word.lower()
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%d" % ("total", 1)
        if word in stopWords:
            print >> sys.stderr, "reporter:counter:Wiki stats,Stop words,%d" % 1
            print "%s\t%d" % ("stop", 1)
        

Overwriting mapper.py


In [57]:
%%writefile reducer.py

import sys
import collections

counter = collections.Counter()

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    counter[key] += count
    
if counter["total"] > 0:
    print "%f" % (100.0*counter["stop"]/counter["total"])

Overwriting reducer.py


In [58]:
! wc -l /datasets/stop_words_en.txt


319 /datasets/stop_words_en.txt


In [59]:
%%bash

OUT_DIR="stopwords_result"
NUM_REDUCERS=1

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="Streaming StopWords" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py,/datasets/stop_words_en.txt \
    -mapper "python mapper.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -cat ${OUT_DIR}/part-00000 | head

41.602622	


17/11/23 18:31:19 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/11/23 18:31:19 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/11/23 18:31:20 INFO mapred.FileInputFormat: Total input files to process : 1
17/11/23 18:31:20 INFO mapreduce.JobSubmitter: number of splits:2
17/11/23 18:31:20 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1511394121436_0025
17/11/23 18:31:20 INFO impl.YarnClientImpl: Submitted application application_1511394121436_0025
17/11/23 18:31:20 INFO mapreduce.Job: The url to track the job: http://be195d52a7ef:8088/proxy/application_1511394121436_0025/
17/11/23 18:31:20 INFO mapreduce.Job: Running job: job_1511394121436_0025
17/11/23 18:31:26 INFO mapreduce.Job: Job job_1511394121436_0025 running in uber mode : false
17/11/23 18:31:26 INFO mapreduce.Job:  map 0% reduce 0%
17/11/23 18:31:42 INFO mapreduce.Job:  map 25% reduce 0%
17/11/23 18:31:48 INFO mapreduce.Job:  map 37% reduce 0%
17/11/23 18:31:54 INFO 